In [ ]:
%matplotlib inline
# Import Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Read the CSV into a Pandas DataFrame
intake_csv = "Resources/Austin_Animal_Center_Intakes.csv"
outcome_csv= "Resources/Austin_Animal_Center_Outcomes.csv"

intake_df = pd.read_csv(intake_csv)
outcome_df = pd.read_csv(outcome_csv)

#convert date variable to datetime format

intake_df['DateTime'] = pd.to_datetime(intake_df['DateTime']).dt.date
outcome_df['DateTime'] = pd.to_datetime(outcome_df['DateTime']).dt.date

#update intake file with column reflecting number of intakes
#and select outcomes for animals with 1 or 2 intakes
intake_counts=intake_df.groupby(['Animal ID']).size().reset_index(name='counts')
intake_df=pd.merge(intake_df, intake_counts, on="Animal ID", how="outer")
reduced_intake_df=intake_df.query('counts <3')

#update outtake file with column reflecting number of intakes per animal
#and select outcomes for animals with 1 or 2 intakes
outcome_df=pd.merge(outcome_df, intake_counts, on="Animal ID", how="left")
reduced_outcome_df=outcome_df.query('counts <3')
#reduced_outcome_df=reduced_outcome_df.drop(columns=['counts'])

In [ ]:
outcome_df

In [ ]:
reduced_outcome_df

In [ ]:
#now take single intake animals and match with any available outcome
reduced_intake_only_one_df=reduced_intake_df.query('counts <2')
#reduced_intake_only_one_df=reduced_intake_only_one_df.add_prefix('IN_')
reduced_outcome_df=reduced_outcome_df.add_prefix("OUT_")

reduced_intake_only_one_df=reduced_intake_only_one_df.rename(index=str,columns={"IN_Animal ID": "Animal ID"})
reduced_outcome_df=reduced_outcome_df.rename(index=str,columns={"OUT_Animal ID": "Animal ID"})
one_intake_merged=pd.merge(reduced_intake_only_one_df, reduced_outcome_df, on="Animal ID", how="left")

In [ ]:
one_intake_merged["num_of_intakes_for_ID"]=1

In [ ]:
#now take two intake animals and figure out order of intakes and outcomes
reduced_intake_two_df=reduced_intake_df.query('counts !=1')


In [ ]:
reduced_intake_two_first_date_lookup_df=reduced_intake_two_df.groupby('Animal ID').aggregate(['min']).reset_index()
reduced_intake_two_first_date_lookup_df = reduced_intake_two_first_date_lookup_df[['Animal ID','DateTime']]
reduced_intake_two_first_date_lookup_df["stay_number"]=1
reduced_intake_two_first_date_lookup_df.columns = reduced_intake_two_first_date_lookup_df.columns.droplevel(1)


reduced_intake_two_first_date_lookup_df.head()

In [ ]:
reduced_intake_two_second_date_lookup_df=reduced_intake_two_df.groupby('Animal ID').aggregate(['max']).reset_index()
reduced_intake_two_second_date_lookup_df = reduced_intake_two_second_date_lookup_df[['Animal ID','DateTime']]
reduced_intake_two_second_date_lookup_df["stay_number"]=2
reduced_intake_two_second_date_lookup_df.columns = reduced_intake_two_second_date_lookup_df.columns.droplevel(1)

In [ ]:
reduced_intake_two_second_date_lookup_df.head()

In [ ]:
multiple_intakes_order_lookup = reduced_intake_two_first_date_lookup_df.append(reduced_intake_two_second_date_lookup_df)

In [ ]:
#multiple_intakes_order_lookup

In [ ]:
#reduced_intake_two_df

reduced_intake_two_df=pd.merge(reduced_intake_two_df, multiple_intakes_order_lookup, on=["Animal ID","DateTime"], how="left")
#reduced_intake_two_df

In [ ]:
#look at outcome records for the two-intake group and determine order

In [ ]:
#reduced_outcome_df

In [ ]:

reduced_outcome_two_first_date_lookup_df=reduced_outcome_df.groupby('Animal ID').aggregate(['min']).reset_index()
reduced_outcome_two_first_date_lookup_df = reduced_outcome_two_first_date_lookup_df[['Animal ID','OUT_DateTime']]
reduced_outcome_two_first_date_lookup_df["stay_number"]=1
reduced_outcome_two_first_date_lookup_df.columns = reduced_outcome_two_first_date_lookup_df.columns.droplevel(1)


reduced_outcome_two_first_date_lookup_df.head()

In [ ]:
reduced_outcome_df=pd.merge(reduced_outcome_df, reduced_outcome_two_first_date_lookup_df, on=["Animal ID","OUT_DateTime"], how="left")

reduced_outcome_df

In [ ]:
reduced_outcome_df['stay_number']=reduced_outcome_df['stay_number'].fillna(2)
#reduced_outcome_df

In [ ]:
#reduced_intake_two_df=reduced_intake_two_df.add_prefix('IN_')
reduced_intake_two_df=reduced_intake_two_df.rename(index=str,columns={"IN_Animal ID": "Animal ID","IN_stay_number":"stay_number"})
reduced_intake_two_df

In [ ]:
#merge the multiple intake with coressponsing outcomes - will not yield two outcomes for animals still in residence
two_intakes_merged=pd.merge(reduced_intake_two_df, reduced_outcome_df, on=["Animal ID", "stay_number"], how="left")
two_intakes_merged["num_of_intakes_for_ID"]="2"

In [ ]:
two_intakes_merged

In [ ]:
one_intake_merged

In [ ]:
working_base_master=one_intake_merged.append(two_intakes_merged, ignore_index=True)

In [ ]:
working_base_master.head()

In [ ]:
working_base_master['time_to_exit'] = working_base_master['OUT_DateTime']-working_base_master['DateTime']

In [ ]:
working_base_master['time_to_exit']=working_base_master['time_to_exit'] / np.timedelta64(1, 'D')

In [ ]:
working_base_master['time_to_exit'].dtype

In [ ]:
working_base_master.head()

In [ ]:
working_base_master["Intake Type"].value_counts()

In [ ]:
#Discover maximum time_to_exit
working_base_master["time_to_exit"].max()

In [ ]:
#number of repeat visits
working_base_master["num_of_intakes_for_ID"].value_counts()

In [ ]:
working_base_master["Intake Type"].value_counts()

In [ ]:
############################

In [ ]:
################################### alternate way to calculate the days 7 14 30 30+ etc for no name.

In [ ]:
#Create dataframe of not named pets at outcome by dropping no names
no_name_df = test_df.fillna({'OUT_Name':'None'})
no_name_df

In [ ]:
no_name_df

In [ ]:
#Confirm NaN values renamed as none
no_name_df["OUT_Name"].value_counts()

In [ ]:
#filter to just unnamed animals
no_name_df = no_name_df.loc[no_name_df["OUT_Name"] == "None", :]

no_name_df.head()

In [ ]:
#Confirm none is the only remaining value
no_name_df["OUT_Name"].value_counts()

In [ ]:
no_name_df["Color"].value_counts()

In [ ]:
no_name_df["OUT_Outcome Type"].value_counts()

In [ ]:
no_name_df.count()

In [ ]:
no_name_df["OUT_Outcome Subtype"].count()

In [ ]:
#filter out outcome types besides adoption
no_name_df = no_name_df.loc[no_name_df["OUT_Outcome Type"] == "Adoption", :]

no_name_df.head()

In [ ]:
#Confirm all outcome types removed besides adoption
no_name_df["OUT_Outcome Type"].value_counts()

In [ ]:
no_name_df = no_name_df = no_name_df.fillna({'OUT_Outcome Subtype':'None'})

In [ ]:
#See what adoption subtypes remain
no_name_df["OUT_Outcome Subtype"].value_counts()

In [ ]:
no_name_df = no_name_df.loc[no_name_df["OUT_Outcome Subtype"] == "None", :]

no_name_df.head()

In [ ]:
no_name_df.count()

In [ ]:
no_name_df_all = no_name_df[no_name_df['time_to_exit'].between(0, 2000)]
no_name_df_all

In [ ]:
no_name_count_all = no_name_df_all['time_to_exit'].count()
no_name_count_all

In [ ]:
no_name_df_7 = no_name_df[no_name_df['time_to_exit'].between(0, 7)]
no_name_df_7

#df = df[df['closing_price'].between(99, 101)]

In [ ]:
no_name_count_7 = no_name_df_7['time_to_exit'].count()
no_name_count_7

In [ ]:
no_name_df_8_14 = no_name_df[no_name_df['time_to_exit'].between(8, 14)]
no_name_df_8_14

In [ ]:
no_name_count_8_14 = no_name_df_8_14['time_to_exit'].count()
no_name_count_8_14

In [ ]:
no_name_df_15_30 = no_name_df[no_name_df['time_to_exit'].between(15, 30)]
no_name_df_15_30

In [ ]:
no_name_count_15_30 = no_name_df_15_30['time_to_exit'].count()
no_name_count_15_30

In [ ]:
no_name_df_over_30 = no_name_df[no_name_df['time_to_exit'].between(31, 2000)]
no_name_df_over_30

In [ ]:
no_name_count_over_30 = no_name_df_over_30['time_to_exit'].count()
no_name_count_over_30

In [ ]:
#next calculate the percentages for 7, 14, 30, and over 30 for name_df

In [ ]:
no_name_count_all

In [ ]:
no_name_count_7

In [ ]:
no_name_count_8_14

In [ ]:
no_name_count_15_30

In [ ]:
no_name_count_over_30

In [ ]:
no_name_count_7+no_name_count_8_14+no_name_count_15_30+no_name_count_over_30

In [ ]:
#percent adopted within 7 days
pct_no_name_adopted_7 = no_name_count_7/no_name_count_all
pct_no_name_adopted_7

In [ ]:
#percent adopted 8-14
pct_no_name_adopted_8_14 = no_name_count_8_14/no_name_count_all
pct_no_name_adopted_8_14

In [ ]:
#percent adopted 15-30
pct_no_name_adopted_15_30 = no_name_count_15_30/no_name_count_all
pct_no_name_adopted_15_30

In [ ]:
#percent adopted 30
pct_no_name_adopted_over_30 = no_name_count_over_30/no_name_count_all
pct_no_name_adopted_over_30

In [ ]:
# Build Pie Chart

# Set Pie Chart Variables
range_type = ["7 days", "8-14 days", "15-30 days", "Over 30 days"]
name_adoption_pct = [pct_name_adopted_7, pct_name_adopted_8_14, pct_name_adopted_15_30, pct_name_adopted_over_30]
colors = ["#E28683", "#F9D649", "#96CDF6", "#F9D649"]
explode = (0.1,0,0)

# Tell matplotlib to put the pie chart together
plt.pie(name_adoption_pct, labels=range_type, colors=colors, autopct="%1.1f%%", shadow=True, startangle=280)

# Create a title for the pie chart
plt.title("% Days Until Adoption (Named Animals)")

# Save an image of our chart
plt.savefig("../Images/% Days Until Adoption (Named Animals).png")

# Show Figure
plt.show()

In [ ]:
# Build Pie Chart

# Set Pie Chart Variables
range_type = ["7 days", "8-14 days", "15-30 days", "Over 30 days"]
no_name_adoption_pct = [pct_no_name_adopted_7, pct_no_name_adopted_8_14, pct_no_name_adopted_15_30, pct_no_name_adopted_over_30]
colors = ["#E28683", "#F9D649", "#96CDF6", "#F9D649"]
explode = (0.1,0,0)

# Tell matplotlib to put the pie chart together
plt.pie(no_name_adoption_pct, labels=range_type, colors=colors, autopct="%1.1f%%", shadow=True, startangle=280)

# Create a title for the pie chart
plt.title("% Days Until Adoption (Non-Named Animals)")

# Save an image of our chart
plt.savefig("../Images/% Days Until Adoption (Non-Named Animals).png")

# Show Figure
plt.show()

In [ ]:
# Build Pie Chart

# Set Pie Chart Variables
no_name_adoption_chart = [no_name_per_capita_transfer, no_name_per_capita_adopted, no_name_per_capita_return_to_owner]
x_axis = np.arange(len(no_name_adoption_chart))
plt.bar(x_axis, no_name_adoption_chart, color='b', alpha=0.5, align="center")
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, ["Transfered", "Adopted", "Returned To Owner"])
plt.title("Outcomes Per 100 Unnamed Animals")
plt.xlabel("Outcomes")
plt.ylabel("Number of Occurances")

# Save an image of our chart
plt.savefig("../Images/Adoption Per Capita (Named Animals).png")

# Show Figure
plt.show()

In [ ]:
import matplotlib.patches as mpatches

#Build Bar Chart

# Values of each group
no_name_chart = [no_name_per_capita_transfer, no_name_per_capita_adopted, no_name_per_capita_return_to_owner]
name_chart = [name_per_capita_transfer, name_per_capita_adopted, name_per_capita_return_to_owner]

tick_locations = [1, 2, 3]

# Create brown bars
plt.bar(tick_locations, no_name_chart, color='#217fd1', alpha=0.5, align="center")

# Create brown bars
plt.bar(tick_locations, name_chart, bottom=no_name_chart, color='#21d1b3', alpha=0.5, align="center")

# Create green bars (middle), on top of the firs ones
#plt.bar(x_axis, name_chart, bottom=no_name_chart, color='#21d1b3', width=barWidth)
# Create green bars (top)
#plt.bar(x_axis, bars3, bottom=bars, color='#2d7f5e', edgecolor='white', width=barWidth)


#combined_adoption_chart = [no_name_per_capita_transfer, name_per_capita_transfer, no_name_per_capita_adopted, name_per_capita_adopted, no_name_per_capita_return_to_owner, name_per_capita_return_to_owner]
x_axis = np.arange(len(combined_adoption_chart))
#plt.bar(x_axis, combined_adoption_chart, color=['#d9e0df', '#d9e0df', '#217fd1', '#21d1b3', '#d9e0df', '#d9e0df'], alpha=0.5, align="center")
#tick_locations = [value for value in x_axis]

#barlist=plt.bar([no_name_per_capita_transfer, name_per_capita_transfer, no_name_per_capita_adopted, name_per_capita_adopted, no_name_per_capita_return_to_owner, name_per_capita_return_to_owner])
#barlist[0].set_color('#F9D649')

plt.xticks(tick_locations, ["Transfered", "Adopted", "Returned"], rotation=45)
plt.title("Outcomes Per 100 Animals")
plt.xlabel("Outcomes")
plt.ylabel("Number of Occurances")

# Defining the legends values
#plt.legend((name_per_capita_transfer, no_name_per_capita_transfer), ('Named', 'Unnamed'), loc="best")

red_patch = mpatches.Patch(color='#217fd1', label='Unnamed')
plt.legend(handles=[red_patch])

blue_patch = mpatches.Patch(color='#21d1b3', label='Named')
plt.legend(handles=[blue_patch, red_patch])

# Save an image of our chart
plt.savefig("../Images/Adoption Per Capita (Combined).png")

# Show Figure
plt.show()



In [ ]:
# set width of bar
barWidth = 0.25
 
# set height of bar
no_name_chart = [no_name_per_capita_transfer, no_name_per_capita_adopted, no_name_per_capita_return_to_owner]
name_chart = [name_per_capita_transfer, name_per_capita_adopted, name_per_capita_return_to_owner]

# Set position of bar on X axis
r1 = np.arange(len(no_name_chart))
r2 = [x + barWidth for x in r1]
#r2 = [x + barWidth for x in r1]

#tick_locations = [0, 1, 2]
 
# Make the plot
plt.bar(r1, no_name_chart, color='#0414f2', width=barWidth)
plt.bar(r2, name_chart, color='#14c603', width=barWidth)
 
# Add xticks on the middle of the group bars
#plt.xlabel('Outcomes', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(no_name_chart))], ['Transfered', 'Adopted', 'Return To owner'])

#plt.xticks(tick_locations, ["Transfered", "Adopted", "Returned To Owner"])
plt.title("Outcomes Per 100 Animals", fontweight='bold')
plt.xlabel("Outcomes", fontweight='bold')
plt.ylabel("Number of Occurances", fontweight='bold')



red_patch = mpatches.Patch(color='#217fd1', label='Unnamed')
plt.legend(handles=[red_patch])

blue_patch = mpatches.Patch(color='#21d1b3', label='Named')
plt.legend(handles=[blue_patch, red_patch])

plt.show()


In [ ]:
# set width of bar
barWidth = 0.25
 
# set height of bar
#bars1 = [12, 30, 1, 8, 22]
#bars2 = [28, 6, 16, 5, 10]
#bars3 = [29, 3, 24, 25, 17]

# set height of bar
no_name_chart = [no_name_per_capita_transfer, no_name_per_capita_adopted, no_name_per_capita_return_to_owner]
name_chart = [name_per_capita_transfer, name_per_capita_adopted, name_per_capita_return_to_owner]

ind = np.arange(len(no_name_chart))

# Set position of bar on X axis
#r1 = np.arange(len(bars1))
#r2 = [x + barWidth for x in r1]
#r3 = [x + barWidth for x in r2]

r1 = np.arange(len(no_name_chart))
r2 = [x + barWidth for x in r1]
#r3 = [x + barWidth for x in r2]
 
# Make the plot
#plt.bar(r1, bars1, color='#7f6d5f', width=barWidth, edgecolor='white', label='var1')
#plt.bar(r2, bars2, color='#557f2d', width=barWidth, edgecolor='white', label='var2')
#plt.bar(r3, bars3, color='#2d7f5e', width=barWidth, edgecolor='white', label='var3')

plt.bar(r1, no_name_chart, color='#217fd1', width=barWidth, edgecolor='white', label='Unnamed')
plt.bar(r2, name_chart, color='#21d1b3', width=barWidth, edgecolor='white', label='Named')
#plt.bar(r3, bars3, color='#2d7f5e', width=barWidth, edgecolor='white', label='var3')


# Add xticks on the middle of the group bars
#plt.xlabel('Outcomes', fontweight='bold')
#plt.xticks([r + barWidth for r in range(len(no_name_chart))], ['Transfered', 'Adopted', 'Returned To Owner'])

plt.ylabel('Number of Occurances')
plt.xlabel('Outcomes')
plt.title('Outcomes Per 100 Animals')
plt.xticks(ind)
plt.xticklabels(('Transfered', 'Adopted', 'Returned To Owner'))
plt.legend()

#plt.title("Outcomes Per 100 Animals", fontweight='bold')
#plt.xlabel("Outcomes", fontweight='bold')
#plt.ylabel("Number of Occurances", fontweight='bold')

# Create legend & Show graphic
#plt.legend()
plt.show()


In [ ]:
# set width of bar
barWidth = 0.25
 
# set height of bar
transfered_chart = [no_name_per_capita_transfer, name_per_capita_transfer]
adopted_chart = [no_name_per_capita_adopted, name_per_capita_adopted]
return_to_owner_chart = [no_name_per_capita_return_to_owner, name_per_capita_return_to_owner]
 
# Set position of bar on X axis
r1 = np.arange(len(transfered_chart))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
 
# Make the plot
plt.bar(r1, transfered_chart, color='#217fd1', width=barWidth, edgecolor='white', label='Transfered')
plt.bar(r2, adopted_chart, color='#ed9b04', width=barWidth, edgecolor='white', label='Adopted')
plt.bar(r3, return_to_owner_chart, color='#21d1b3', width=barWidth, edgecolor='white', label='Returned To Owner')

#21d1b3

# Add xticks on the middle of the group bars
plt.xlabel('Outcomes', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(transfered_chart))], ['Unnamed', 'Named'])
plt.ylabel('Number of Occurances', fontweight='bold')
plt.title('Outcomes Per 100 Animals', fontweight='bold')

# Create legend & Show graphic
plt.legend()
plt.grid(axis='y', alpha=.20)
plt.savefig("Outcomes Unnamed Vs. Named.png", dpi=1020)
plt.show()

In [ ]:
# set width of bar
barWidth = 0.25
 
# set height of bar
per_capita_non_pit_bull = [all_non_pit_bull_return_per_capita]
per_capita_pit_bull = [all_pit_bull_return_per_capita]
#return_to_owner_chart = [no_name_per_capita_return_to_owner, name_per_capita_return_to_owner]
 
# Set position of bar on X axis
r1 = np.arange(len(all_non_pit_bull_return_per_capita))
r2 = [x + barWidth for x in r1]
#r3 = [x + barWidth for x in r2]
 
# Make the plot
plt.bar(r1, per_capita_non_pit_bull, color='#217fd1', width=barWidth, edgecolor='white', label='Transfered')
plt.bar(r2, per_capita_pit_bull, color='#ed9b04', width=barWidth, edgecolor='white', label='Adopted')
#plt.bar(r3, return_to_owner_chart, color='#21d1b3', width=barWidth, edgecolor='white', label='Returned To Owner')

#21d1b3

# Add xticks on the middle of the group bars
plt.xlabel('Outcomes', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(per_capita_non_pit_bull))], ['Owner Surrender'])
plt.ylabel('Number of Occurances', fontweight='bold')
plt.title('Outcomes Per 100 Animals', fontweight='bold')
# Create legend & Show graphic
plt.legend()
plt.grid(axis='y', alpha=.20)
plt.savefig("Outcomes Pitbull vs. Non-Pitbull.png", dpi=1020)
plt.show()